In [2]:
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
from IPython.display import display, Markdown, Latex
import numpy as np

import glob
frac = 0.01
# path ='./data/RFReg_' # use your path
path ='./../data/Pobreza/pobreza_' # use your path

allFiles = []
allFiles += glob.glob(path+str(frac)+'_q*.csv')
allFiles = sorted(allFiles)
allFiles

['./../data/Pobreza/pobreza_0.01_q2010-02-15.csv',
 './../data/Pobreza/pobreza_0.01_q2010-05-15.csv',
 './../data/Pobreza/pobreza_0.01_q2010-08-15.csv',
 './../data/Pobreza/pobreza_0.01_q2010-11-15.csv',
 './../data/Pobreza/pobreza_0.01_q2011-02-15.csv',
 './../data/Pobreza/pobreza_0.01_q2011-05-15.csv',
 './../data/Pobreza/pobreza_0.01_q2011-08-15.csv',
 './../data/Pobreza/pobreza_0.01_q2011-11-15.csv']

In [4]:
cpi_M = pd.read_csv('https://raw.githubusercontent.com/matuteiglesias/IPC-Argentina/main/data/info/indice_precios_M.csv', index_col=0)
cpi_M.index = pd.to_datetime(cpi_M.index)

cpi_Q = pd.read_csv('https://raw.githubusercontent.com/matuteiglesias/IPC-Argentina/main/data/info/indice_precios_Q.csv', index_col=0)
cpi_Q.index = pd.to_datetime(cpi_Q.index)

cpi_actual = cpi_M.loc['2022-05']['index'][0]
cpi_2010Q4 = cpi_Q.loc['2010-11']['index'][0]
cpi_ref = cpi_M.loc['2016-01']['index'][0]

In [5]:
aglo_labels = pd.read_csv('./../data/info/aglo_labels.csv')

In [27]:
# df_list = []
# for f in allFiles:
#     print(f)
    
f = './../data/Pobreza/pobreza_0.01_q2010-11-15.csv'
groupers = ['AGLOMERADO', 'PROV', 'NOMPROV']
vars_ = ['INGRESO', 'P47T_persona']

data = pd.read_csv(f, usecols = ['Q'] + groupers + vars_)
data = data.merge(aglo_labels)
data['AGLO_si'] = data.AGLOMERADO != 0
data['Total'] = True

data = data.loc[data.AGLO_si == 1]

data.loc[data.INGRESO == 0, 'decil_P47T'] = 'sin ingresos'
data.loc[data.INGRESO == 1, 'P47T_persona'] = data.loc[data.INGRESO == 1, 'P47T_persona'] + np.random.uniform(0, .01, data.INGRESO.sum())
data.loc[data.INGRESO == 1, 'decil_P47T'] = pd.qcut(data.loc[data.INGRESO == 1, 'P47T_persona'], q = 10, labels = range(1, 11))

data['Q'] = pd.to_datetime(data['Q'])
# df104 = df.loc[df.Q == '2010-11-15']

In [28]:
# df104 = df.loc[df.Q == '2010-11-15']

In [29]:
# pd.read_csv(f, nrows = 5).T

In [30]:
## Tabla:

# Personas con ingresos (aprox 14.377, sin ingresos 10.510 para un total de 24.887 en aglos)


In [31]:
tabla = data.groupby(['decil_P47T']).agg({'P47T_persona': ['min', 'max', 'size', 'sum', 'mean']})['P47T_persona']
tabla[['min', 'max', 'sum', 'mean']] = tabla[['min', 'max', 'sum', 'mean']]*(cpi_2010Q4/cpi_ref)
tabla[['size', 'sum']] = 100*tabla[['size', 'sum']]
tabla['pct'] = 100*tabla['sum']/tabla['sum'].sum()
tabla.loc['Total'] = tabla.iloc[:-1].agg({'min': 'min', 'max': 'max','size': 'sum', 'sum': 'sum', 'mean': 'mean', 'pct': 'sum'})

tabla[['min', 'max', 'sum', 'mean']] = tabla[['min', 'max', 'sum', 'mean']].round(-1).astype(int)
tabla['size'] = (tabla['size']/1e3).round(1)
tabla['sum'] = (tabla['sum']/1e6).round(1)
tabla['pct'] = tabla['pct'].round(1)
tabla.columns = ['Mínimo', 'Máximo', 'Población (miles)', 'Ingresos (mill. ARS)', 'Media', '% de suma de ingresos']
tabla

,Mínimo,Máximo,Población (miles),Ingresos (mill. ARS),Media,% de suma de ingresos
decil_P47T,,,,,,
1,30,560,1542.2,528.9,340,1.5
2,560,840,1542.1,1091.0,710,3.0
3,840,1130,1542.2,1528.4,990,4.2
4,1130,1410,1542.1,1979.7,1280,5.5
5,1410,1880,1542.2,2568.5,1670,7.1
6,1880,2250,1542.1,3088.1,2000,8.6
7,2250,2810,1542.1,3791.3,2460,10.5
8,2810,3380,1542.2,4604.1,2990,12.8
9,3380,4690,1542.1,6091.8,3950,16.9


In [32]:
36102.7/15421.5

2.341062801932367

In [26]:
15421.5 / 14387.85

1.0718418665749225